In [385]:
import requests
from bs4 import BeautifulSoup
import json
import time
from datetime import datetime
import pandas as pd
import os

header = {'User-Agent': 'mdemiceli@gmail.com'}

In [555]:
os.chdir(r'C:\Users\nicho\Downloads')
cik_csv = pd.read_csv('cik_ticker.csv', sep = '|')
cik_nums = cik_csv.iloc[:, 0:3]

for i in range(len(cik_nums['CIK'])):
    if len(str(cik_nums['CIK'][i])) != 10:
        diff = 10 - len(str(cik_nums['CIK'][i]))
        cik_num[i] = '0' * diff + str(cik_nums['CIK'][i])
        
cik_num[100]

'0001035632'

In [519]:
cik = cik_num[13000]

base_submissions_url = r'https://data.sec.gov/submissions/CIK' + cik + r'.json'

content = requests.get(base_submissions_url, headers=header)
decoded_content = content.json()

submissions_json_file = open(r'sec_' + cik + r'.json', 'w')
json.dump(decoded_content, submissions_json_file, indent=3)

with open(r'sec_' + cik + r'.json') as sec:
    submissions_mass = json.load(sec)
    
print(base_submissions_url)

https://data.sec.gov/submissions/CIK0000895530.json


In [520]:
forms = submissions_mass['filings']['recent']['form'] 
filing_dates = submissions_mass['filings']['recent']['filingDate']
filing_dates_datetime = []
for date in filing_dates:
    date_time = pd.to_datetime(date, infer_datetime_format=True)
    filing_dates_datetime.append(date_time)

    
accession_numbers = submissions_mass['filings']['recent']['accessionNumber']
accession_numbers_clean = []
for number in accession_numbers:
    accession_numbers_clean.append(number.replace('-', ''))
    
    
submission_data = {'FilingDates': filing_dates_datetime, 'AccessionNumber':  accession_numbers_clean, 'Form': forms}
submission_data

{'FilingDates': [Timestamp('2006-08-28 00:00:00'),
  Timestamp('2006-06-22 00:00:00'),
  Timestamp('2006-03-20 00:00:00'),
  Timestamp('2006-02-07 00:00:00'),
  Timestamp('2005-12-29 00:00:00'),
  Timestamp('2005-10-24 00:00:00'),
  Timestamp('2005-10-21 00:00:00'),
  Timestamp('2005-10-05 00:00:00'),
  Timestamp('2005-09-27 00:00:00'),
  Timestamp('2005-09-23 00:00:00'),
  Timestamp('2005-09-16 00:00:00'),
  Timestamp('2005-08-25 00:00:00'),
  Timestamp('2005-08-23 00:00:00'),
  Timestamp('2005-08-23 00:00:00'),
  Timestamp('2005-08-12 00:00:00'),
  Timestamp('2005-06-29 00:00:00'),
  Timestamp('2005-06-28 00:00:00'),
  Timestamp('2005-05-11 00:00:00'),
  Timestamp('2005-03-23 00:00:00'),
  Timestamp('2005-02-01 00:00:00'),
  Timestamp('2004-12-29 00:00:00'),
  Timestamp('2004-12-28 00:00:00'),
  Timestamp('2004-11-29 00:00:00'),
  Timestamp('2004-09-27 00:00:00'),
  Timestamp('2004-08-27 00:00:00'),
  Timestamp('2004-08-23 00:00:00'),
  Timestamp('2004-08-18 00:00:00'),
  Timestamp('

In [521]:
print(len(submission_data['FilingDates']))
print(len(submission_data['AccessionNumber']))
print(len(submission_data['Form']))

108
108
108


In [522]:
submissions_df = pd.DataFrame(data=submission_data)
submissions_df

,FilingDates,AccessionNumber,Form
0,2006-08-28,000095013706009576,N-PX
1,2006-06-22,000113673006000081,NSAR-A
2,2006-03-20,000095013706003274,N-8F
3,2006-02-07,000113673006000007,NSAR-B/A
4,2005-12-29,000113673005000147,NSAR-B
...,...,...,...
103,1995-06-29,000089109395001418,NSAR-A
104,1995-06-29,000089109395001405,N-30D
105,1995-04-14,000095012495001119,DEF 14A
106,1994-12-29,000089109394002533,NSAR-B


In [523]:
accession_ten_k = submissions_df['AccessionNumber'][submissions_df['Form'] == '10-K']
accession_ten_k

Series([], Name: AccessionNumber, dtype: object)

In [524]:
base_url = r'https://www.sec.gov/Archives/edgar/data/'

accession = r'000119312509258790'

filing_url = base_url + cik + '/' + accession + '/index.json'

filing_content = requests.get(filing_url, headers=header).json()

for file in filing_content['directory']['item']:
    
    # grab the filing summary and create a new url leading to the file so we can download it
    if file['name'] == 'FilingSummary.xml':
        
        xml_summary = r'https://www.sec.gov' + filing_content['directory']['name'] + '/' + file['name']
        
        print('-'*100)
        print('File Name: ' + file['name'])
        print('File Path: ' + xml_summary)
print(filing_url)   

"""
Code runs good up until you try to access an accession number that is before 2012. It fluctuates
around this time but it seems like most companies you can't access their FilingSummaryxml before then. 

"""

https://www.sec.gov/Archives/edgar/data/0000895530/000119312509258790/index.json


In [511]:
base_url = xml_summary.replace('FilingSummary.xml', '')
content = requests.get(xml_summary, headers=header).content
soup = BeautifulSoup(content, 'lxml')

reports = soup.find('myreports')

master_reports = []

for report in reports.find_all('report')[:-1]:
    
    report_dict = {}
    report_dict['name_short'] = report.shortname.text
    report_dict['name_long'] = report.longname.text
    report_dict['position'] = report.position.text
    report_dict['category'] = report.menucategory.text
    report_dict['url'] = base_url + report.htmlfilename.text
    
    master_reports.append(report_dict)

    
statements_url = []

for report_dict in master_reports:
    
    item1 = r'Consolidated Balance Sheets'.upper()
    item2 = r'Consolidated Statements of Operations'.upper()
    item3 = r'Consolidated Statements of Cash Flows'.upper()
    
    report_list = [item1, item2, item3]
    
    if report_dict['name_short'] in report_list:
        print('-'*100)
        print(report_dict['name_short'])
        print(report_dict['url'])
        
print(statements_url)

[]
